In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
import joblib

In [2]:
reg_train=pd.read_csv("/kaggle/input/playground-series-s4e4/train.csv")
reg_test=pd.read_csv("/kaggle/input/playground-series-s4e4/test.csv")
sample_submit=pd.read_csv("/kaggle/input/playground-series-s4e4/sample_submission.csv")
reg_train.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [3]:
encoder = LabelEncoder()
reg_train['Sex'] = encoder.fit_transform(reg_train['Sex'])
reg_test['Sex'] = encoder.transform(reg_test['Sex'])
reg_train.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,0,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,1,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,2,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,1,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [4]:
x_train = reg_train[["Sex", "Length", "Diameter", "Height", "Whole weight", "Whole weight.1", "Whole weight.2", "Shell weight"]]
y_train = reg_train['Rings']
x_test = reg_test[["Sex", "Length", "Diameter", "Height", "Whole weight", "Whole weight.1", "Whole weight.2", "Shell weight"]]

In [5]:
array_X = np.array(x_train)
array_y = np.array(y_train)
array_X_test = np.array(x_test)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(array_X, array_y, test_size=0.2, random_state=42)
xg_model = xgb.XGBRegressor()
xg_model.fit(X_train, y_train)
joblib.dump(xg_model, "xg_model.pkl")
xg_model_1 = joblib.load('xg_model.pkl')
y_pred = xg_model_1.predict(X_test)
rmsle_score = np.sqrt(mean_squared_log_error(y_test, y_pred))
print("RMSLE:", rmsle_score)

RMSLE: 0.1532314696918464


In [7]:
y_pred = xg_model_1.predict(array_X_test)
y_pred

array([ 9.762816 ,  9.5608835, 10.304032 , ..., 14.482715 , 13.414977 ,
        9.157325 ], dtype=float32)

In [8]:
# F=0, I=1, M=2
#lv = [0, 0.550, 0.430, 0.150,	0.7715,	0.3285,	0.1465,	0.2400]
lv = [2, 0.595, 0.475, 0.150, 0.9145, 0.3755, 0.2055, 0.2500]
array_values = np.array(lv)
xg_model_1.predict([array_values])

array([10.244803], dtype=float32)

In [9]:
sub=sample_submit[['id']]
sub['Rings'] = y_pred
sub.to_csv('./submission.csv', index=False)